Calculate proportional change in genetic map distance (instead of absolute) (ran in R 4.2.1)

In [ ]:
# list of linkage groups
ilist <- 1:32
ilist2 <- ilist[-c(16,25,29,31)]
# list of chromosomes
jlist <- 1:24
jlist1 <- jlist[-16]
jlist2 <- append(jlist1, c("26", "27", "28", "1A", "4A"))

# loop
mapply(function(i, j) {
  # import file
  setwd("path/")
  lep <- read.delim(Sys.glob(paste0("order-physical-",i,"-*.txt")), header = FALSE)
  # manipulate file
  lep2 <- lep[-(1:3),(1:3)]
  colnames(lep2) <- c("marker", "male", "female")
  lep2$male <- as.numeric(lep2$male)
  lep2$female <- as.numeric(lep2$female)
  lep2$phy <- lepPos2$pos[match(lep2$marker,lepPos2$marker)]
  lep2$order <- 1:nrow(lep2)
  maleMap <- tail(lep2$male, n=1)
  femaleMap <- tail(lep2$female, n=1)
  lep2$maleFlip <- lep2$male
  lep2$femaleFlip <- lep2$female
  lep2$maleProp <- lep2$maleFlip/maleMap
  lep2$femaleProp <- lep2$femaleFlip/femaleMap
  lep3 <- lep2[,c(1,4,8,9)]
  lep4 <- melt(lep3, id.vars = c("marker", "phy") , variable.name = "sex")
}, 
i = ilist2,
j = jlist2,
SIMPLIFY = FALSE)

Apply a zero-or-one inflated beta regression (zoib) regression model to infer associations between heterochiasmy with gene density (example given here) and distance to nearest chromosome end 

In [ ]:
library(brms)

# import list of intervals
int4

# describe model
zoib_ng <- bf(
  hetAbsRescale ~ numGenes, #mean
  phi ~ numGenes, #precision
  zoi ~ numGenes, #zero-one inflation aka alpha
  coi ~ numGenes, #condiation one-inflation aka gamma
  family = zero_one_inflated_beta()
)

# run model and save output to R data file
fit <- brm(
  formula = zoib_ng,
  data = int4,
  cores = 4,
  file = "brm-zoib-hetAbsRescale_numGenes"
)

# load saved R data file
fit <- readRDS("brm-zoib-hetAbsRescale_numGenes.rds")

fit

# see posterior samples
posterior_samples(fit)

# transform posterior samples then summarise
posterior_samples(fit, pars = "b_")[,1:4] %>% 
  mutate_at(c("b_phi_Intercept"), exp) %>% 
  mutate_at(vars(-"b_phi_Intercept"), plogis) %>% 
  posterior_summary() %>% 
  as.data.frame() %>% 
  rownames_to_column("Parameter") %>% 
  kable(digits = 2) 
# effect of predictor variable
posterior_samples(fit, pars = "b_")[,5:8] %>% 
  mutate_at(c("b_phi_numGenes"), exp) %>% 
  mutate_at(vars(-"b_phi_numGenes"), plogis) %>% 
  posterior_summary() %>% 
  as.data.frame() %>% 
  rownames_to_column("Parameter") %>% 
  kable(digits = 2) 

mean(int4$hetAbsRescale %in% 0:1) %>% round(3)

mean(int4$hetAbsRescale[int4$hetAbsRescale %in% 0:1] == 1) %>% 
  round(3)

# plot estimated mean (mu parameter) & 95% CI against raw data points
c_eff <- conditional_effects(fit, dpar = "mu")
c_eff_df <- as.data.frame(c_eff$numGenes)
colnames(c_eff_df) <- c("numGenes", "hetAbsRescale", "cond", "effect1", "estimate", "se", "lower", "upper")

corr_hetVsnumGenes <- ggplot() +
  geom_ribbon(data = c_eff_df, aes(ymin=lower, ymax=upper, x=effect1), fill = "#bdb4a870") +
  geom_line(data = c_eff_df, aes(x = effect1, y = estimate), col = "#40372A") +
  geom_point(data = int4, aes(x = numGenes, y = hetAbsRescale, col = type), shape = 19, size = 1, alpha = 0.5) +
  scale_color_manual(name = "Heterochiasmy type", values = c("#DC3220", "#005AB5", "#A19584"), labels = c("female-bias\n(n=295)", "male-bias\n(n=386)", "no heterochiasmy\n(n=218)")) +
  theme_classic() + theme(panel.grid = element_blank()) +
  labs(x = "Number of genes (gene density)",
       y = "|rescaled heterochiasmy index|")
